### This file querries all 300K-ish CVE entires in the NVD and store the top 3 matches into mongo.

In [ ]:
### Include all necessary imports ###

import sys
import json
from pathlib import Path
from pymongo import MongoClient
import copy


import copy

### Get the correct directory resolved for imports ###
project_root = Path().resolve()
while not (project_root / 'embedding_pipeline').exists() and project_root != project_root.parent:
    project_root = project_root.parent

sys.path.append(str(project_root))

from embedding_pipeline.weaviate_db.weaviate_config import create_remote_weaviate_client
from embedding_pipeline.weaviate_db.weaviate_config import verify_weaviate_client_ready
from embedding_pipeline.weaviate_db.weaviate_config import close_weaviate_client
from embedding_pipeline.weaviate_db.weaviate_config import list_weaviate_collections, inspect_collection_properties, retrieve_existing_weaviate_collection
from embedding_pipeline.weaviate_db.weaviate_query_operations import query_weaviate_collection
from embedding_pipeline.weaviate_db.weaviate_query_operations import get_query_vector_responses


from embedding_pipeline.embedding_models.nomic_embed import embed_prompt_with_nomic
from embedding_pipeline.embedding_models.DISTIL_BERT_embed import embed_prompt_with_distil_bert
from embedding_pipeline.embedding_models.SBERT_mini_lm_l6_embed import embed_prompt_with_sbert_mini_l6
from embedding_pipeline.embedding_models.SBERT_mini_lm_l12_embed import embed_prompt_with_sbert_mini_l12
from embedding_pipeline.embedding_models.GTE_large_embed import embed_prompt_with_gte_large

### Load CVE CPE json data ###
with open("../../enriched_cve_data/cve-cpe-foss_data.json", "r", encoding="utf-8") as f:
    cve_cpe_json_data = json.load(f)

In [ ]:
### Get the local weavite collection name ###
WEAVIATE_COLL_NAME = "FOSS_vectors"

### Connect to remote weavite client ###
remote_client = create_remote_weaviate_client()

remote_foss_collection = retrieve_existing_weaviate_collection(collection_name=WEAVIATE_COLL_NAME,
                                                              weaviate_client=remote_client)




### Connect to MongoDB clinet ###

uri = "mongodb://localhost:27017"
mongodb_client = MongoClient(uri)


try:
    mongodb_client.admin.command("ping")
    print("Connected to MongoDB!")
except :
    print("❌ Could not connect to MongoDB:")

### Get the CVE database ###
NVD_CVE_DB = mongodb_client["nvdcve"]

### Get / create the collection ###
enriched_cve_collection = NVD_CVE_DB["enriched_cve_entries"]



In [ ]:

### set the batch size ###
BATCH_SIZE = 500
BATCH = []


for entry in cve_cpe_json_data:
    try:
        # all processing code here

        enriched_mongo_entry = copy.deepcopy(entry)

        ### Temporary list to hold vector responses for individual CPEs (ven:product combos)
        enriched_cpe_matches = []


        if 'cpes_with_versions' not in entry or not entry['cpes_with_versions']:
            continue

        for cpe in entry['cpes_with_versions']:


            
            ### Get the CPE identifier ###
            vendor_query: str = cpe
            print(f"Embedding vendor query: {vendor_query}")

            nomic_vec: list[float] = embed_prompt_with_nomic(prompt=vendor_query)
            
            distil_vec: list[float] = embed_prompt_with_distil_bert(prompt=vendor_query)
            sbert_l6_vec: list[float]= embed_prompt_with_sbert_mini_l6(prompt=vendor_query)
    
            sbert_l12_vec: list[float] = embed_prompt_with_sbert_mini_l12(prompt=vendor_query)
            gte_large_vec: list[float] = embed_prompt_with_gte_large(prompt= vendor_query)


            ### Get the query object ###
            nomic_query_return_remote = query_weaviate_collection(vector_query=nomic_vec,
                                               target_name_vector_query="ollama_nomic_name_vec",
                                               weaviate_collection=remote_foss_collection)
            distil_query_return_remote = query_weaviate_collection(vector_query= sbert_l6_vec, #####REVERSED ######
                                                            target_name_vector_query="distil_bert_name_vec",
                                                            weaviate_collection=remote_foss_collection)
            sbert_l6_query_return_remote = query_weaviate_collection(vector_query=distil_vec, ##### REVERSED #####
                                                            target_name_vector_query="sbert_minilm_l6_v2_name_vec",
                                                            weaviate_collection=remote_foss_collection)

            sbert_l12_query_return_remote = query_weaviate_collection(vector_query=sbert_l12_vec,
                                                            target_name_vector_query="sbert_minilm_l12_v2_name_vec",
                                                            weaviate_collection=remote_foss_collection)
            gte_large_query_return_remote = query_weaviate_collection(vector_query=gte_large_vec,
                                                               target_name_vector_query="gte_large_name_vec",
                                                              weaviate_collection=remote_foss_collection)

            ### Get the import vector info ###
            nomic_response = get_query_vector_responses(response=nomic_query_return_remote)
            distil_response = get_query_vector_responses(response=distil_query_return_remote)
            sbert_l6_response = get_query_vector_responses(response=sbert_l6_query_return_remote)
            sbert_l12_response = get_query_vector_responses(response=sbert_l12_query_return_remote)
            gte_large_response = get_query_vector_responses(response=gte_large_query_return_remote)
            
        

            
            ### Getting the first vector because this will have the highest cosine similarity / distance  ### 
            # nomic_final = nomic_response[0]
            # distil_final = distil_response[0]
            # sbert_l6_final = sbert_l6_response[0]
            # sbert_12_final = sbert_l12_response[0]
            # gte_large_final = gte_large_response[0]
            
            enriched_cpe_matches.append({
                "cpe": vendor_query,
                "nomic": nomic_response,
                "distil": distil_response,
                "sbert_l6": sbert_l6_response,
                "sbert_l12": sbert_l12_response,
                "gte_large": gte_large_response
            })

        
        ### An 'enriched_mongo_entry' represents 1 entire CVE (that has multiple cpes) ###
        enriched_mongo_entry["foss_vec_matches"] = enriched_cpe_matches
        
        
        
        BATCH.append(enriched_mongo_entry)

        # Batch insert every 500 entries
        if len(BATCH) >= BATCH_SIZE:
            enriched_cve_collection.insert_many(BATCH)
            BATCH = []


    except Exception as e:
        print(f"Failed on entry {entry.get('id', 'unknown')}: {e}")
        continue
        
# Insert any remaining
if BATCH:
    enriched_cve_collection.insert_many(BATCH)
    
    

